In [ ]:
!pip install pdfplumber
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 38

## 데이터 클리닝

1. PDF 데이터 체크
2. 방법론

  * 불필요한 페이지 제거 -> 정규식 사용하여 제거 (적용)
  * 형태소 토큰화를 통해서 의미적으로 필요한 토큰만을 사용할 예정 (미적용)
  * 의미적인 단위로 줄 바꿈 현재 논문의 줄바꿈 단위랑 비슷한 것 같긴 한데 정확하게 원하던 단위인지는 확신 불가(적용?)
3. 검증 (미적용)
  * 토큰화 없이 유사도 기반으로 데이터 서치
    * tf-idf embedding 기반 유사도 서치 결과
    * sbert-embedding 기반 유사도 결과
  * 형태소 토큰화 적용하고 유사도 기반으로 데이터서치
    * tf-idf embedding 기반 유사도 서치 결과
    * sbert-embedding 기반 유사도 결과



In [ ]:
## tf idf
import pandas as pd
from copy import deepcopy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from collections import deque

# Sample documents
train_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/도배하자_QA/train.csv')
train_csv = train_csv[train_csv['category'].isin(('마감하자', '시공', '타 마감하자'))]
qa_set_1 = train_csv[['질문_1', '답변_3']].apply(lambda x: x[0] + x[1], axis=1).to_list()
qa_set_2 = train_csv[['질문_2', '답변_3']].apply(lambda x: x[0] + x[1], axis=1).to_list()

document = qa_set_1 + qa_set_2

# document = train_csv['질문_1'].to_list() + train_csv['질문_2'].to_list()

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(document)
def find_most_close_one(query, documents):

  answers = tfidf_vectorizer.transform(documents)
  query_vec = tfidf_vectorizer.transform([query])

  answers_samples, answers_features = answers.shape

  most_close_one = None
  most_close_one_score = 0000

  for i in range(answers_samples):
    if cosine_similarity(query_vec, answers[i]) >= most_close_one_score:
      most_close_one = i
      most_close_one_score = cosine_similarity(query_vec, answers[i])

  return documents[most_close_one], most_close_one_score

def find_top_k_most_close_one(query, documents, k=3):

  # tf - idf로 벡터화
  answers = tfidf_vectorizer.transform(documents)
  query_vec = tfidf_vectorizer.transform([query])


  answers_samples, answers_features = answers.shape

  most_close_one = None
  most_close_one_score = 0000

  #queue
  queue = deque(maxlen=k)
  queue.appendleft(most_close_one_score)

  #find top-k closest ones
  for i in range(answers_samples):
    if cosine_similarity(query_vec, answers[i]) >= most_close_one_score:
      most_close_one_score = cosine_similarity(query_vec, answers[i])
      queue.appendleft((documents[i],most_close_one_score))

  return list(queue)

In [ ]:
import pdfplumber
import re

# pdf에서 의미있는 텍스트만을 추출
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        ref_page = 9999
        # 본문 페이지
        meaningful_page_pattern = r'-(\d+)-|- (\d+) -'
        # 참고 문헌 페이지부터는 추출 중단
        refernence_pattern = r'\s*참\s*고\s*문\s*헌\s*'

        for num, page in enumerate(pdf.pages):
            # Extract text from each page
            page_text = page.extract_text()

            page_text_has_pattern = re.search(meaningful_page_pattern, page_text)

            #subtract page after refernce page
            page_text_has_refernence = True if re.match(refernence_pattern, page_text) else False
            ref_page = num if page_text_has_refernence else ref_page

            if page_text_has_pattern and num < ref_page:
              # Concatenate text from all pages
              text += page_text + '\n'

    return text


def text_keep_sentence_splitter(text, max_chunk=100, overlap=0):
  chunk_start = 0

  chunks = []
  while chunk_start < len(text):
    #firstly make the chunk
    chunk = text[chunk_start:chunk_start + max_chunk]

    #find the last point of sentences
    last_punctuation = max(
          chunk.rfind("."),
          chunk.rfind("?"),
          chunk.rfind("!"),
          chunk.rfind("\n"),
      )
    # chunk again based on the last punctuation
    chunk = text[chunk_start:last_punctuation]

    #make overlap
    overlap_text = text[last_punctuation - overlap : last_punctuation]

    #adjust the departure of chunk
    chunk_start += last_punctuation - overlap + min(
        overlap_text.find("."),
        overlap_text.find("?"),
        overlap_text.find("!"),
        overlap_text.find("\n")
    )
  return chunks

qa_refernce_pdf = '/content/drive/MyDrive/Colab Notebooks/dacon/도배하자_QA/공동주택의 공정별 하자발생 및 예방대책에 관한 연구.pdf'

pdf_text = extract_text_from_pdf(qa_refernce_pdf)
# print(pdf_text)




Ⅰ. 서 론
1. 연구배경 및 목적
공동주택은 1980년대 이후 도시인구의 급증과 경제 성장에 따라 계속적으로
공급되면서 주요 주택유형의 하나로 자리 잡았다. 근래에 들어 대량의 공동주
택 특히 아파트의 공급이 증가하면서 수요는 점차적으로 고품질화ㆍ다양화돼
가고 있으며, 공동주택의 비중이 높아지고 있다. 1987년 주택보급을 위한 주택
건설촉진법 개정안이 통과되고, 계속해서 공동주택건설이 서민의 내 집 마련의
방법으로 이용됨으로서, 많은 공동주택이 지어져 왔다. 작년 말 기준 주택보급
률 추정치는 전국 107.1%로, 재작년말에 비해 1.2%포인트 상승했으며, 수도권
은 96.9%, 서울은 91.3%로 서울이 사상 최초로 90%대의 주택보급률에 진입하게
되었다1).
공동주택의 건설량이 전체 주택건설량에서 차지하는 비율은 계속해서 증가하
고 있다. 그러나 공동주택의 품질면에서는 소득 증가와 더불어 생활양식의 다
변화와 함께 주택상품의 품질에 대한 관심과 요구가 높아지고 있으며, 건축기
술 수준향상과 더불어 건축자재의 고급화ㆍ양산화ㆍ규격화가 이루어졌음에도
불구하고 사용상ㆍ기능상ㆍ미관상ㆍ안전상 지장을 초래할 정도의 기능상의 하
자 및 부실공사는 공동주택 공급량에 비례하여 끊임없이 발생하고 있어 입주자
들의 품질에 대한 요구수준을 충족시키지 못했다. 공동주택이 일반서민의 생
활터전이자 삶의 활력소와 휴식공간으로 자리잡아 가고 있으며, 건강하고 쾌적
한 환경을 요구하고 있다. 이에 결함의 발생은 주거생활의 불편을 초래하고 소
비자에게 정신적ㆍ경제적 피해를 주게 된다. 또한, 주거성능 및 기능을 저해하
는 요소로 아파트의 공급자와 수요자 사이에서 이에 대한 책임소재를 놓고 갈
등을 일으키게 된다. 즉, 주택상품이 지니고 있는 결함에 대해 보수 및 수정을
요구하는 소비자와 결함이 아니라고 주장하는 공급자간에 분쟁이 발생하게 된
다. 이는 결국 아파트 품질에 대해 공급자와 수요자 사이에서 하자분쟁의 형태
로 나타나게 되며, 수요자에게는 주거환경 저하를, 공급자에게는 민원발생

In [ ]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=250)
texts = text_splitter.split_text(pdf_text)
for i, text in enumerate(texts):
    # print(f'doc: #{i}', text)
    if i == 5:
      break

# print(len(texts))

doc: #0 Ⅰ. 서 론
1. 연구배경 및 목적
공동주택은 1980년대 이후 도시인구의 급증과 경제 성장에 따라 계속적으로
공급되면서 주요 주택유형의 하나로 자리 잡았다. 근래에 들어 대량의 공동주
택 특히 아파트의 공급이 증가하면서 수요는 점차적으로 고품질화ㆍ다양화돼
가고 있으며, 공동주택의 비중이 높아지고 있다. 1987년 주택보급을 위한 주택
건설촉진법 개정안이 통과되고, 계속해서 공동주택건설이 서민의 내 집 마련의
방법으로 이용됨으로서, 많은 공동주택이 지어져 왔다. 작년 말 기준 주택보급
률 추정치는 전국 107.1%로, 재작년말에 비해 1.2%포인트 상승했으며, 수도권
은 96.9%, 서울은 91.3%로 서울이 사상 최초로 90%대의 주택보급률에 진입하게
되었다1).
공동주택의 건설량이 전체 주택건설량에서 차지하는 비율은 계속해서 증가하
고 있다. 그러나 공동주택의 품질면에서는 소득 증가와 더불어 생활양식의 다
변화와 함께 주택상품의 품질에 대
doc: #1 입하게
되었다1).
공동주택의 건설량이 전체 주택건설량에서 차지하는 비율은 계속해서 증가하
고 있다. 그러나 공동주택의 품질면에서는 소득 증가와 더불어 생활양식의 다
변화와 함께 주택상품의 품질에 대한 관심과 요구가 높아지고 있으며, 건축기
술 수준향상과 더불어 건축자재의 고급화ㆍ양산화ㆍ규격화가 이루어졌음에도
불구하고 사용상ㆍ기능상ㆍ미관상ㆍ안전상 지장을 초래할 정도의 기능상의 하
자 및 부실공사는 공동주택 공급량에 비례하여 끊임없이 발생하고 있어 입주자
들의 품질에 대한 요구수준을 충족시키지 못했다. 공동주택이 일반서민의 생
활터전이자 삶의 활력소와 휴식공간으로 자리잡아 가고 있으며, 건강하고 쾌적
한 환경을 요구하고 있다. 이에 결함의 발생은 주거생활의 불편을 초래하고 소
비자에게 정신적ㆍ경제적 피해를 주게 된다. 또한, 주거성능 및 기능을 저해하
는 요소로 아파트의 공급자와 �
doc: #2 �고 있으며, 건강하고 쾌적
한 환경을 요구하고 있다. 이에 결함의 발생은 주거생활의 불편을 초래하

In [ ]:
test_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/도배하자_QA/test.csv')
test_csv.head()

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?


In [ ]:

import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")
collection.add(
    documents=texts,
    ids=[f"id_{i}" for i in range(len(texts))]
)

In [ ]:
queries = test_csv['질문'].to_list()
query = queries[55]
document_query = document[15]
print(query)
print(document_query)

원목마루의 어떤 단점이 있는지 알려주세요. 그리고 도배지가 남으면 어떻게 처리해야 하나요?
실크벽지의 얼룩은 어떻게 지워?실크벽지의 얼룩을 지우려면 부드러운 천을 물에 적신 후 얼룩이 있는 곳을 가볍게 닦아내세요. 이때 강한 압력을 가하지 않도록 주의하고, 너무 급하게 마찰하지 않도록 주의하세요. 만약 이 방법으로 얼룩이 사라지지 않는다면, 실크벽지의 표면 손상을 막고자 하는 목적으로 설계된 세탁액을 사용하여 조금씩 묻힌 천으로 얼룩을 가볍게 닦아보세요. 그래도 얼룩이 사라지지 않는다면 실크벽지를 교체하는 것이 좋습니다.


In [ ]:
collection.query(
    query_texts=[document[15]],
    n_results=3,
)

{'ids': [['id_47', 'id_103', 'id_83']],
 'distances': [[0.14543722569942474,
   0.14567410945892334,
   0.15864335000514984]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['�\n제3조 공동주택의 담보책임 및 하자보수에 관하여는 제46조의 개\n정규정을 적용한다.\n□ 적용 범위\n○ 집합건물법\n가. 집합건물법은 구분소유관계가 인정될 수 있는 집합건물을 대상\n으로 하고, 분양자의 담보책임을 규율하고 있다. 그리고 하자담\n보책임의 내용은 민법상 수급인의 담보책임에 관한 규정을 준용\n하도록 하고 있다(동법 제9조).\n나. “집합건물”은 상가, 오피스텔 외 공동주택을 모두 포함하는 개\n념이다. 따라서, 공동주택의 경우에도 집합건물법이 적용되는 것\n이 원칙이겠으나, 2005년 5월 26일 주택법 및 집합건물법이 함께\n개정되면서 공동주택의 하자담보책임에 관하여는 집합건물법의\n적용은 배제되었다(2005년 주택법 및 집합건물법 개정 전에는 사\n업주체에 대한 관계에서 집합건물법이 우선 적용되었다).\n다. 집합건물법은 “분양자”의 담보책임을 규정하고 있으므로, 분\n양계약의 당사자가 아닌 단순한 시공자는 수분양자에 대해 담보\n책임을 부담하',
   ' 그에 준하도록 한다.\n이상과 같이 종합적인 관점에서 실수를 미연에 막기 위한 규칙을 만들어 그\n규칙을 지키게 하고, 하도급업체도 참가시킨 품질의 자주관리체제를 취해야 한\n다. 만일 문제발생 시 에는 문제점을 철저히 분석하여 실수의 재발방지책을 세\n워, 곧 실행에 옮기게 하는 체제구성이 필요하다.\n(3) 시공자 측면에서의 방지대책\n하자의 원인은 역시 설계, 시공, 재료 및 사용의 모든 부분에 걸치고 있으며\n시공과 재료에서 보면, 특히 그 원인에도 이중적인 성격을 가지고 있다고 할\n수 있다.\n예를 들면,‘타일의 

In [ ]:
print(query)
collection.query(
    query_texts=[query],
    n_results=3,
)

원목마루의 어떤 단점이 있는지 알려주세요. 그리고 도배지가 남으면 어떻게 처리해야 하나요?


{'ids': [['id_15', 'id_9', 'id_104']],
 'distances': [[0.7727813720703125, 0.8048259019851685, 0.8120258450508118]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['며 그것이 하자라는 의미이다.\n(1) 하자의 정의 개념에서 본 사용검사 전 하자\n하자의 정의를 어떻게 할 것인가가 문제가 될 수 있지만 건축물이 일단 완\n공되었다는 전제로 건축물의 완공 이전에 존재하였던 하자는 채무불이행이고\n완공 이후에 존재한 것은 하자라고 보는 것 같다. 여기서 완공의 개념은 사\n용검사와는 차이가 있을 수 있으 나 사용검사가 있으면 완공되었다고 보고 있\n- 6 -\n으므로 완공시점을 사용승인시점과 같다고 잠정적으로 가정할 수 있을 것이\n다.\n이러한 정의에 의할 때 완공 이전에 발생한 것은 하자가 아니므로 하자보수\n청구권의 대상이 아니라는 주장이 있을 수 있다. 그러나 동산의 경우 완전한\n물건으로서 인도를 하고 나서 인도된 물건을 검사하여 하자가 있으면 하자로\n인한 감액이나 기타 청구권을 인정하고 있는데, 이것은 인도받기 전에 이미\n발생한 완전한 상태와의 차이에 대하여 하자로서 인정하고 있다는 것을 의\n미한다.\n상법 제69조 제1항에',
   '�계도면이나 시방서대로 건축하지 않은 부실공사\n와 관련법규와 설계도서 등에 의하여 시공을 하였다 하더라도 시공이 잘못되어\n건축물이 지녀야할 고유한 기능을 발휘하지 못하는 것 외에 미관상, 안전상 지\n장을 초래하는 모든 경우도 포함된다고 할 것이다.\n국어사전에서는 “법률 또는 당사자가 예상한 완전한 상태나 조건 따위가 결\n여되어 있는 상태”로 정의한다. 하자를 결함(缺陷)이라고도 말할 수 있으나\n결함은 “부족하거나 완전하지 못하여 안전성이 결여된 것” 이므로 법적인 분\n쟁거리로 발전하지 않는 상태라고 할 수 있다. 법원이나 민법에서

In [ ]:
queries = test_csv['질문'].to_list()

print(queries[55])

find_top_k_most_close_one(queries[55], texts)

원목마루의 어떤 단점이 있는지 알려주세요. 그리고 도배지가 남으면 어떻게 처리해야 하나요?



[('�\n제3조 공동주택의 담보책임 및 하자보수에 관하여는 제46조의 개\n정규정을 적용한다.\n□ 적용 범위\n○ 집합건물법\n가. 집합건물법은 구분소유관계가 인정될 수 있는 집합건물을 대상\n으로 하고, 분양자의 담보책임을 규율하고 있다. 그리고 하자담\n보책임의 내용은 민법상 수급인의 담보책임에 관한 규정을 준용\n하도록 하고 있다(동법 제9조).\n나. “집합건물”은 상가, 오피스텔 외 공동주택을 모두 포함하는 개\n념이다. 따라서, 공동주택의 경우에도 집합건물법이 적용되는 것\n이 원칙이겠으나, 2005년 5월 26일 주택법 및 집합건물법이 함께\n개정되면서 공동주택의 하자담보책임에 관하여는 집합건물법의\n적용은 배제되었다(2005년 주택법 및 집합건물법 개정 전에는 사\n업주체에 대한 관계에서 집합건물법이 우선 적용되었다).\n다. 집합건물법은 “분양자”의 담보책임을 규정하고 있으므로, 분\n양계약의 당사자가 아닌 단순한 시공자는 수분양자에 대해 담보\n책임을 부담하',
  array([[0.02319507]])),
 ('대로 시공하지 않아 비틀림이 있다면 비틀림이 있다는 사실, 즉\n창문으로서 제대로 기능하지 못하고 있다는 점이 하자가 된다. 다만, 미시공,\n오시공이 있다면 그 시공의 목적이 되는 기능은 잃어버렸다고 할 것이고 이러\n한 점에서 오시공만 입증하면 설계 당시의 기대되었던 건축물과 다른 상태가\n되었다는 사실, 즉 하자가 있다는 사실에 대하여도 사실상 주장 입증한 것이나\n다름없다.\n아파트 쪽의 주장에서의 미시공, 오시공은 하자의 원인행위를 의미하면서 동\n시에 그로 인한 기능, 미관, 안전상의 하자를 주장하는 의미이다. 예를 들어,\n전유부분의 패널히팅부의 미시공, 오시공이란 것은 결국 그로 인하여 난방기능\n에 장애가 온 상태가 존재하며 그것이 하자라는 의미이다.\n(1) 하자의 정의 개념에서 본 사용검사 전 하자\n하자의 정의를 어떻게 할 것인가가 문제가 될 수 있지만 건축물이 일단 완\n공되었다는 전제로 건축물의 완공 이전

In [ ]:
print(document[15], '\n')
find_top_k_most_close_one(document[15], texts)

실크벽지의 얼룩은 어떻게 지워?실크벽지의 얼룩을 지우려면 부드러운 천을 물에 적신 후 얼룩이 있는 곳을 가볍게 닦아내세요. 이때 강한 압력을 가하지 않도록 주의하고, 너무 급하게 마찰하지 않도록 주의하세요. 만약 이 방법으로 얼룩이 사라지지 않는다면, 실크벽지의 표면 손상을 막고자 하는 목적으로 설계된 세탁액을 사용하여 조금씩 묻힌 천으로 얼룩을 가볍게 닦아보세요. 그래도 얼룩이 사라지지 않는다면 실크벽지를 교체하는 것이 좋습니다. 



[('�택에서 발생될 수 있는 공정별 하자의\n유형 및 특징을 파악하고, 계속해서 발생하는 하자와 그 원인을 규명하여 반복\n적으로 하자가 발생되지 않도록 공동주택 하자예방을 위한 자료를 제시하는 것\n을 궁극적인 목적으로 한다.\n- 2 -\n2. 연구범위 및 방법\n본 연구는 공동주택을 대상으로 각 공정별로 발생하는 하자에 대한 분석과\n하자 발생 원인과 문제점을 파악하고, 그 결과로서 각 반복적으로 하자가 발생\n하는 공정별 종합대책을 모색하고자 다음과 같은 방법으로 진행하였다.\n연구범위는 최근 입주한 서울지역 아파트 1,910세대를 대상으로 시공회사에\n서 조사ㆍ분석한 하자보수대장을 근거로 하였다. 공동주택 하자관리 분야 중\n공정별 하자 유형을 중심으로 분석하고, 다음으로 선정된 하자 항목들에서 공\n동주택 하자분류체계에 맞추어 공정별로 재분류하였다.\n연구방법은 공동주택 하자보수에 관한 이론적 고찰은 이와 관련된 각종',
  array([[0.06844327]])),
 (' 및 시공으로\n1) 건설교통부 주택보급률(시도별) 통계자료. 2006.12\n- 1 -\n하자 발생을 최소화해야 하는 것이 필수적이나 아파트가 다양한 자재 및 부재\n의 복합재 라는 특성으로 인해 하자 발생은 필연적인 것으로 판단되며, 기존\n하자보수제도의 효율적인 운용과 아파트의 기능 및 성능의 향상, 소비자와 공\n급자간의 하자발생에 따른 갈등을 해결하기 위해 아파트에서 발생한 결함을 신\n속히 조사ㆍ분석하여 하자여부를 평가하는 것이 필요하다.\n따라서 지속적으로 발생되는 하자의 경우는 설계ㆍ자재 및 시공ㆍ관리등 각\n각의 측면에서 근본적인 하자원인을 검토ㆍ분석하여 그 방지 대책을 수립할 필\n요가 있다. 이에 본 연구에서는 공동주택에서 발생될 수 있는 공정별 하자의\n유형 및 특징을 파악하고, 계속해서 발생하는 하자와 그 원인을 규명하여 반복\n적으로 하자가 발생되지 않도록 공동주택 하자예방을 위한 자료를 제시하는 것\n을 궁극적인',
  array([[0.03949991

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 32.5MiB/s]
